## 1. Google Drive 마운트

In [42]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. 패키지 설치

In [1]:
!pip install openai==1.97.1 streamlit faiss-cpu scikit-learn pandas numpy openpyxl pyngrok -q
print("✅ 패키지 설치 완료")

✅ 패키지 설치 완료


## 3. 작업 디렉토리 설정 및 데이터 확인

In [2]:
import os

# ⚠️ 본인의 프로젝트 경로로 수정하세요
PROJECT_PATH = '/Users/choihj/PycharmProjects/FirstFin'
DATA_PATH = PROJECT_PATH + '/Data'

# 작업 디렉토리 변경
os.chdir(PROJECT_PATH)
print(f"📁 작업 디렉토리: {os.getcwd()}")

# 데이터 파일 확인
print(f"\n📊 Data 폴더 파일 목록:")
for f in os.listdir(DATA_PATH):
    print(f"  - {f}")

📁 작업 디렉토리: /Users/choihj/PycharmProjects/FirstFin

📊 Data 폴더 파일 목록:
  - card_product_info_최신.xlsx
  - .DS_Store
  - customers_with_id.csv
  - customer_satisfaction.csv
  - customer_logs.csv
  - deposit_product_info_최신.xlsx
  - deposit_subscriptions.csv
  - card_transactions_updated.csv


## 4. ngrok 설정

In [3]:
# ⚠️ 본인의 ngrok 토큰으로 수정하세요
# https://dashboard.ngrok.com/get-started/your-authtoken
!ngrok config add-authtoken "your ngrok api"
print("✅ ngrok 설정 완료")

Authtoken saved to configuration file: /Users/choihj/Library/Application Support/ngrok/ngrok.yml
✅ ngrok 설정 완료


## 5. app.py 생성 (프로젝트 폴더에 저장)

In [4]:
# app.py 생성 확인
!ls -la App.py

-rw-r--r--@ 1 choihj  staff  42234 Dec  2 19:29 App.py


## 6. Streamlit 실행

In [5]:
from pyngrok import ngrok

# 🔴 기존 ngrok 터널 모두 종료
ngrok.kill()
print("⛔ 기존 ngrok 터널 종료 완료")

⛔ 기존 ngrok 터널 종료 완료


In [6]:
import os
import subprocess
from pyngrok import ngrok

# 경로 이동
PROJECT_PATH = '/Users/choihj/PycharmProjects/FirstFin'
os.chdir(PROJECT_PATH)
print(f"📁 이동 완료: {os.getcwd()}")

# 새로운 포트 사용 (8502 사용)
port = 8502

# ngrok 새 터널 실행
public_url = ngrok.connect(port).public_url
print(f"\n🌐 접속 주소: {public_url}")

# Streamlit 실행
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", str(port), "--server.headless", "true"],
    stdout=open("streamlit.log", "w"),
    stderr=subprocess.STDOUT
)
print(f"✅ Streamlit 실행 중 (PID: {process.pid})")

📁 이동 완료: /Users/choihj/PycharmProjects/FirstFin

🌐 접속 주소: https://preveniently-noncensored-roxann.ngrok-free.dev
✅ Streamlit 실행 중 (PID: 3349)


In [18]:
from pyngrok import ngrok
tunnels = ngrok.get_tunnels()
print(tunnels)

[<NgrokTunnel: "https://preveniently-noncensored-roxann.ngrok-free.dev" -> "http://localhost:8502">]


## 7. (선택) 추천 시스템 직접 테스트

In [ ]:
# hybrid_recommender.py 직접 테스트
from hybrid_recommender import HybridFinzRecommender
import pandas as pd

# 데이터 로드
customers = pd.read_csv(DATA_PATH + '/customers_with_id.csv')
card_info = pd.read_excel(DATA_PATH + '/card_product_info_최신.xlsx')
deposit_info = pd.read_excel(DATA_PATH + '/deposit_product_info_최신.xlsx')
log_df = pd.read_csv(DATA_PATH + '/customer_logs.csv')
satisfaction_df = pd.read_csv(DATA_PATH + '/customer_satisfaction.csv')

# 엔진 초기화
OPENAI_API_KEY = "your api key here"

engine = HybridFinzRecommender(
    customers, card_info, deposit_info,
    log_df, satisfaction_df,
    openai_api_key=OPENAI_API_KEY
)

# 추천 실행
result = engine.full_recommendation('C000001', top_k=3)

print("\n📊 추천 결과:")
for i, rec in enumerate(result['recommendations'], 1):
    print(f"{i}. {rec['product_name']} ({rec['product_type']})")

print("\n📝 추천 설명:")
print(result['explanation'])